In [1]:
import pandas as pd
import dateutil.relativedelta as datediff
# from subprocess import InvalidIndexError

In [2]:
df_bouncing_data= pd.read_excel("/home/rami/Downloads/lk.xlsx",sheet_name='Bouncing Data')

In [3]:
df_DPD_data= pd.read_excel("/home/rami/Downloads/lk.xlsx",sheet_name='DPD')

In [8]:
df_bouncing_data['Presentation_Date']=df_bouncing_data.Presentation_Date.apply(lambda dt:dt.replace(day=5))
df_DPD_data['MONTH']=df_DPD_data.MONTH.apply(lambda dt:dt.replace(day=5))

In [19]:
print(df_bouncing_data['Presentation_Date'].head())
print(df_DPD_dat['MONTH'].head())

Index(['Deal_No', 'dpd', 'max_dpd_quarter', 'max_dpd_half_year',
       'max_dpd_annual'],
      dtype='object')

# DPD part -1 : calculation of dpd querterly etc.,

In [6]:
#calculation maximum due days in the last three months
#calculation max dpds in the last quarted, half yearly, annualy
def max_due_days(row,index,df_DPD_data):
    month1=index[1]-datediff.relativedelta(months=1)
    month2=index[1]-datediff.relativedelta(months=2)
    month3=index[1]-datediff.relativedelta(months=3)
    month4=index[1]-datediff.relativedelta(months=4)
    month5=index[1]-datediff.relativedelta(months=5)
    month6=index[1]-datediff.relativedelta(months=6)
    month7=index[1]-datediff.relativedelta(months=7)
    month8=index[1]-datediff.relativedelta(months=8)
    month9=index[1]-datediff.relativedelta(months=9)
    month10=index[1]-datediff.relativedelta(months=10)
    month11=index[1]-datediff.relativedelta(months=11)
    month12=index[1]-datediff.relativedelta(months=12)
    k=index[0]
    bounces=[0]
    start_date=startdate_dpd.loc[k]['MONTH']
    index1=(index[0],month1)
    index2=(index[0],month2)
    index3=(index[0],month3)
    index4=(index[0],month4)
    index5=(index[0],month5)
    index6=(index[0],month6)
    index7=(index[0],month7)
    index8=(index[0],month8)
    index9=(index[0],month9)
    index10=(index[0],month10)
    index11=(index[0],month11)
    index12=(index[0],month12)
    if month1>=start_date:
        bounces.append(df_DPD_data.loc[index1]['dpd'])
#     print(df_DPD_data.loc[index1]['dpd'])
    if month2>=start_date:
        bounces.append(df_DPD_data.loc[index2]['dpd'])
    if month3>=start_date:
        bounces.append(df_DPD_data.loc[index3]['dpd'])
    if month4>=start_date:
        bounces.append(df_DPD_data.loc[index4]['dpd'])
    if month5>=start_date:
        bounces.append(df_DPD_data.loc[index5]['dpd'])
    if month6>=start_date:
        bounces.append(df_DPD_data.loc[index6]['dpd'])
    if month7>=start_date:
        bounces.append(df_DPD_data.loc[index7]['dpd'])
    if month8>=start_date:
        bounces.append(df_DPD_data.loc[index8]['dpd'])
    if month9>=start_date:
        bounces.append(df_DPD_data.loc[index9]['dpd'])
    if month10>=start_date:
        bounces.append(df_DPD_data.loc[index10]['dpd'])
    if month11>=start_date:
        bounces.append(df_DPD_data.loc[index11]['dpd'])
    if month12>=start_date:
        bounces.append(df_DPD_data.loc[index12]['dpd'])
    return max(bounces[:3]),max(bounces[:6]),max(bounces)

In [11]:
startdate_dpd = pd.DataFrame(df_DPD_data.groupby(['DealId'])['MONTH'].min())
df_DPD_data['max_dpd_quarter'] = None
df_DPD_data['max_dpd_half_year']=None
df_DPD_data['max_dpd_annual']=None
df_DPD_data=df_DPD_data.set_index(['DealId','MONTH'])

In [13]:
# df_DPD_data['max_dpd_in_quarter']=df_DPD_data.apply(lambda row:max_due_days(row,df_DPD_data),axis=1)
for index,row in df_DPD_data.iterrows():
    df_DPD_data.ix[index,'max_dpd_quarter'],df_DPD_data.ix[index,'max_dpd_half_year'],\
    df_DPD_data.ix[index,'max_dpd_annual']=(max_due_days(row,index,df_DPD_data))

In [152]:
df_DPD_data.to_excel('/home/rami/Downloads/dpd2.xlsx')

In [14]:
df_DPD_data.head()

Deal_No  dpd max_dpd_quarter max_dpd_half_year  \
DealId    MONTH                                                        
100897748 2016-02-05    31772    0               0                 0   
          2016-03-05    31772    0               0                 0   
          2016-04-05    31772    0               0                 0   
          2016-05-05    31772    0               0                 0   
          2016-06-05    31772    0               0                 0   

                     max_dpd_annual  
DealId    MONTH                      
100897748 2016-02-05              0  
          2016-03-05              0  
          2016-04-05              0  
          2016-05-05              0  
          2016-06-05              0

## DPD Part 2: Creation of DPDBins

In [21]:
bins=[-1,1,15,30,45,60,75,90,max(df_DPD_data['dpd'])]
labels = ['[0,1]','(1,15]','(15,30]','(30,45]','(45,60]','(60,75]','(75,90]','(90,534]']
df_DPD_data['dpd_bin']=pd.cut(df_DPD_data['dpd'],bins,labels=labels)
df_DPD_data['dpd_bin'].value_counts()

[0,1]       89579
(15,30]      9842
(90,534]     2684
(45,60]      2668
(75,90]      1295
(1,15]        178
(30,45]        45
(60,75]        14
Name: dpd_bin, dtype: int64

# Bouncing data part 1

## deleting the duplicate records in the bouncing data

In [11]:
startdate = pd.DataFrame(df_bouncing_data.groupby(['DealId'])['Presentation_Date'].min())
df_bouncing_data=df_bouncing_data.set_index(['DealId','Deal_No','Presentation_Date'])
from collections import Counter


In [12]:
index_1=df_bouncing_data.index.values

In [13]:
index_2=Counter(index_1)

In [14]:
index_3={}
for key,value in index_2.items():
    if value>1:
        index_3[key]=value

In [15]:
from collections import defaultdict
index_4=defaultdict(list)

In [16]:
df_bouncing_data2=df_bouncing_data.ix[index_3.keys()]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [17]:
for index,row in df_bouncing_data2.iterrows():
    index_4[index].append(row['Clearing_Status'])

In [18]:
df_bouncing_data3=pd.DataFrame(columns=df_bouncing_data2.columns)

In [19]:
for index,row in df_bouncing_data2.iterrows():
    if row['Clearing_Status'] in ['Cleared','Bounced'] and index not in df_bouncing_data3.index:
        df_bouncing_data3=df_bouncing_data3.append(row)

In [95]:
df_bouncing_data2.to_excel('/home/rami/Downloads/bounce_duplicates.xlsx')

In [20]:
# df_bouncing_data.drop(df_bouncing_data.index[], inplace=True)
df_bouncing_data_out=df_bouncing_data.loc[~df_bouncing_data.index.isin(df_bouncing_data2.index.values)]

In [21]:
print(len(df_bouncing_data_out))
len(df_bouncing_data)
df_bouncing_data_out=df_bouncing_data_out.append(df_bouncing_data3)
len(df_bouncing_data_out)

99884


100268

# Bouncing data part 2 - adding no of bounces in quarter etc

In [22]:
def no_of_bounce_in_last_quarter(index,df_bouncing_data):
    month1=index[2]-datediff.relativedelta(months=1)
    month2=index[2]-datediff.relativedelta(months=2)
    month3=index[2]-datediff.relativedelta(months=3)
    month4=index[2]-datediff.relativedelta(months=4)
    month5=index[2]-datediff.relativedelta(months=5)
    month6=index[2]-datediff.relativedelta(months=6)
    month7=index[2]-datediff.relativedelta(months=7)
    month8=index[2]-datediff.relativedelta(months=8)
    month9=index[2]-datediff.relativedelta(months=9)
    month10=index[2]-datediff.relativedelta(months=10)
    month11=index[2]-datediff.relativedelta(months=11)
    month12=index[2]-datediff.relativedelta(months=12)
    k=index[0]
    start_date=startdate.loc[k]['Presentation_Date']
    no_of_bounces=[]
    if month1>=start_date:
        index2=(index[0],index[1],month1)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month2>=start_date:
        index2=(index[0],index[1],month2)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month3>=start_date:
        index2=(index[0],index[1],month3)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month4>=start_date:
        index2=(index[0],index[1],month4)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month5>=start_date:
        index2=(index[0],index[1],month5)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month6>=start_date:
        index2=(index[0],index[1],month6)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month7>=start_date:
        index2=(index[0],index[1],month7)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month8>=start_date:
        index2=(index[0],index[1],month8)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month9>=start_date:
        index2=(index[0],index[1],month9)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month10>=start_date:
        index2=(index[0],index[1],month10)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month11>=start_date:
        index2=(index[0],index[1],month11)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    if month12>=start_date:
        index2=(index[0],index[1],month12)
        try:
            if df_bouncing_data.loc[index2]['Clearing_Status'] =='Bounced':
                no_of_bounces.append(1)
        except TypeError:
            no_of_bounces=no_of_bounces
    return sum(no_of_bounces[:3]),sum(no_of_bounces[:6]),sum(no_of_bounces)

In [23]:
df_bouncing_data_out['no_of_bounces_quarterly']=None
df_bouncing_data_out['no_of_bounces_half_yearly']=None
df_bouncing_data_out['no_of_bounces_annualy']=None
# 
df_bouncing_data_out.head()

,,,Presentation_Receipt_Id,Presenation_Amount,Repay_Mode,Clearing_Status,Bouncing_Reason,Cancellation_reason,quarter_bounced,no_of_bounces_quarterly,no_of_bounces_half_yearly,no_of_bounces_annualy
DealId,Deal_No,Presentation_Date,,,,,,,,,,
100907530,31974,2016-02-05,135156,5130,PDC,Cleared,NaN,NaN,None,None,None,None
100897748,31772,2016-02-05,138503,2488,PDC,Cleared,NaN,NaN,None,None,None,None
100899546,31807,2016-02-05,138520,2160,PDC,Cleared,NaN,NaN,None,None,None,None
100903070,31879,2016-02-05,138549,2347,PDC,Cleared,NaN,NaN,None,None,None,None
100907245,31968,2016-02-05,138580,4101,PDC,Cleared,NaN,NaN,None,None,None,None


In [24]:
for index,row in df_bouncing_data_out.iterrows():
#     print(index)
    df_bouncing_data_out.ix[index,'no_of_bounces_quarterly'],df_bouncing_data_out.ix[index,'no_of_bounces_half_yearly'],\
    df_bouncing_data_out.ix[index,'no_of_bounces_annualy']=(no_of_bounce_in_last_quarter(index,df_bouncing_data_out))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


In [25]:
df_bouncing_data_out.to_excel('/home/rami/Downloads/bounce_quarterly.xlsx')